In [49]:
from dfply import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Manufacturers and Products Table

In [50]:
manu = pd.read_csv("manufacturers.csv")
prod = pd.read_csv("products.csv")

## Print the first five rows of a table

In [51]:
prod >> head(5) # prod %>% head(5)

,prd_id,name,price,mfr_id
0,1,Hard drive,240,5
1,2,Memory,120,6
2,3,ZIP drive,150,4
3,4,Floppy disk,5,6
4,5,Monitor,240,1


## Get a unique list of product names

In [52]:
prod >> distinct(X.name) # prod %>% distinct(name)

,prd_id,name,price,mfr_id
0,1,Hard drive,240,5
1,2,Memory,120,6
2,3,ZIP drive,150,4
3,4,Floppy disk,5,6
4,5,Monitor,240,1
5,6,DVD drive,180,2
6,7,CD drive,90,2
7,8,Printer,270,3
8,9,Toner cartridge,66,3
9,10,DVD burner,180,2


### As you can see, unlike dplyr, dfply.distinct outputs all columns and not just the "name" column

## Print first 5 records of the manufacturer table

In [53]:
manu >> head(5)

,mfr_id,mfr_name
0,1,Sony
1,2,Creative Labs
2,3,Hewlett-Packard
3,4,Iomega
4,5,Fujitsu


## Filter for products with price > 100

In [54]:
prod >> mask(X.price > 100) # prod %>% filter(price > 100)

,prd_id,name,price,mfr_id
0,1,Hard drive,240,5
1,2,Memory,120,6
2,3,ZIP drive,150,4
4,5,Monitor,240,1
5,6,DVD drive,180,2
7,8,Printer,270,3
9,10,DVD burner,180,2


## Merge Product with manufacturers dataset

In [55]:
prod_manu = prod >> left_join(manu, by = "mfr_id") #prod %>% left_join(manu, by = "mfr_id")
prod_manu

,prd_id,name,price,mfr_id,mfr_name
0,1,Hard drive,240,5,Fujitsu
1,2,Memory,120,6,Winchester
2,3,ZIP drive,150,4,Iomega
3,4,Floppy disk,5,6,Winchester
4,5,Monitor,240,1,Sony
5,6,DVD drive,180,2,Creative Labs
6,7,CD drive,90,2,Creative Labs
7,8,Printer,270,3,Hewlett-Packard
8,9,Toner cartridge,66,3,Hewlett-Packard
9,10,DVD burner,180,2,Creative Labs


### Names have to be consistent to join

## Filter for manufacturers with average product price of greater than 150


In [56]:
manu_150 = prod_manu >> group_by(X.mfr_id,X.mfr_name) >> summarize(avg_price = mean(X.price)) >> mask(X.avg_price >= 150)
# prod_manu %>% group_by(mfr_id,mfr_name) %>% summarise(avg_price = mean(price)) %>% filter(avg_price >= 150)

In [57]:
manu_150

,mfr_name,mfr_id,avg_price
0,Sony,1,240.0
1,Creative Labs,2,150.0
2,Hewlett-Packard,3,168.0
3,Iomega,4,150.0
4,Fujitsu,5,240.0


# Baseball Dataframe

In [58]:
baseball = pd.read_csv("baseball.csv",names = ["Player_Name","Team_Name","Bats","Hits"])
baseball >> head(5)

,Player_Name,Team_Name,Bats,Hits
0,A. J. Pierzynski,White Sox,479,133
1,Aaron Hill,Diamondbacks,609,184
2,Adam Dunn,White Sox,539,110
3,Adam Jones,Orioles,648,186
4,Adam LaRoche,Nationals,571,155


## Filter for teams with 7 or more number of players in it

In [59]:
# Create a column with number of players by team in the dataframe
baseball >> group_by(X.Team_Name) >> summarize(player_count = n_distinct(X.Player_Name)) >> \
mask(X.player_count >= 7) 
# baseball %>% group_by(Team_Name) %>% summarise(player_count = n_distinct(X.Player_Name)) 
# %>% ungroup() %>% filter(player_count >= 7)

,Team_Name,player_count
0,Angels,8
6,Cardinals,7
20,Rangers,7
26,Tigers,7
27,Twins,7
28,White Sox,9
29,Yankees,7


### Its a must to pass a column name to n or n_distinct - Need to pass the granularity of the table to obtain frequency

## Compute the batting average, Batting Average = Team_total_hits/ Teams total at bat

In [60]:
#Always remember to ungroup before performing any operations
baseball = baseball >> group_by(X.Team_Name) >> summarize(bat_avg = X.Hits.sum()/X.Bats.sum()) >> ungroup()
#baseball <- baseball %>% group_by(Team_Name) %>% summarize(bat_avg = sum(Hits)/sum(Bats)) %>% ungroup()

baseball >> head(5)

,Team_Name,bat_avg
0,Angels,0.287629
1,Astros,0.289931
2,Athletics,0.253380
3,Blue Jays,0.250563
4,Braves,0.266344


## Filter for teams with batting average greater than .2 and less than .25

In [61]:
baseball >> mask(X.bat_avg >= 0.2,X.bat_avg <= 0.25)
# baseball %>% filter(bat_avg >= 0.2 & bat_avg <= 0.25)

,Team_Name,bat_avg
12,Mariners,0.242055
18,Phillies,0.250000
21,Rays,0.240749
23,Reds,0.249316


# Explore the spread, gather, unite and seperate functions

In [62]:
student_score = pd.read_csv("C:/3-MSBA - Summer/Stat for Data Scientists/Stat Project/student-mat.csv")
student_score >> head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Medu_des,Mjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,Higher,at_home,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,4th Grade,at_home,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,4th Grade,at_home,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,Higher,health,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,Secondary,other,...,4,3,2,1,2,5,4,6,10,10


## Selecting columns using columns_to and columns_from

In [ ]:
student_score = student_score >> select(columns_to(2,inclusive = True),columns_from(-3))
# student_score %>% select(1:3,(length(names(student_score))-2):(length(names(student_score))))
student_score >> head(5)



## Roll up the data by school, sex and age

In [64]:
student_score = student_score >> group_by(X.school,X.sex,X.age) >> summarize(G1_sum = X.G1.sum(), G2_sum = X.G2.sum() , G3_sum = X.G3.sum())

## Gather the dataset for the aggregated grade columns G1_sum,G2_sum and G3_sum

In [65]:
student_score = student_score >> gather('variable','value',['G1_sum','G2_sum','G3_sum'])
#student_score %>% gather("variable","value",G1:G3)

In [66]:
student_score >> head(5)

,age,sex,school,variable,value
0,15,F,GP,G1_sum,382
1,16,F,GP,G1_sum,551
2,17,F,GP,G1_sum,556
3,18,F,GP,G1_sum,312
4,19,F,GP,G1_sum,135


## Spread the gathered columns

In [67]:
student_score >> spread(X.variable,X.value) >> head(5)

,age,sex,school,G1_sum,G2_sum,G3_sum
0,15,F,GP,382,372,363
1,15,M,GP,539,560,560
2,16,F,GP,551,579,569
3,16,M,GP,587,584,578
4,17,F,GP,556,548,531


## Unite function - combining the school and age columns

In [68]:
student_score = student_score >> unite('school_age',X.school,X.age) 
# student_score <- student_score %>% unite(school_age,school,age,sep = '_')  

['school', 'age'] _ True maintain


In [69]:
student_score >> head(5)

,sex,variable,value,school_age
0,F,G1_sum,382,GP_15
1,F,G1_sum,551,GP_16
2,F,G1_sum,556,GP_17
3,F,G1_sum,312,GP_18
4,F,G1_sum,135,GP_19


## Separate function

In [70]:
# You can also custom define the columns to assume NaN values based if the column consists unequal number of elements
student_score = student_score >> separate(X.school_age, ['school','age'],sep = "_",remove = True)
#student_score %>% separate(school_age, c('school','age'))
student_score >> head(5)

,sex,variable,value,school,age
0,F,G1_sum,382,GP,15
1,F,G1_sum,551,GP,16
2,F,G1_sum,556,GP,17
3,F,G1_sum,312,GP,18
4,F,G1_sum,135,GP,19
